# Case-Explainer Tutorial: Iris Dataset

Welcome to this tutorial on **case-based explainability** using the classic Iris dataset!

## What You'll Learn

- How to create case-based explanations for ML predictions
- Understanding the correspondence metric
- Interpreting explanations through similar training examples
- Analyzing prediction confidence via correspondence scores

## What is Case-Based Explainability?

Instead of saying *"Feature X has importance 0.45"*, case-based explainability answers:

> **"This prediction was made because it resembles these 5 training examples"**

This approach is particularly valuable when:
- Domain experts need to verify predictions against known cases
- You want concrete examples rather than abstract feature importances
- The training data has meaningful context worth surfacing

## 1. Setup and Data Loading

First, let's import the required libraries and load the Iris dataset.

In [ ]:
# Import libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Add case_explainer to path
sys.path.insert(0, '..')
from case_explainer import CaseExplainer

# Set random seed
np.random.seed(42)
print("Libraries imported successfully!")

In [ ]:
# Load Iris dataset
data = load_iris()
X = data.data
y = data.target
feature_names = list(data.feature_names)
class_names = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Dataset: {len(X)} total samples")
print(f"Training: {len(X_train)} samples")
print(f"Testing: {len(X_test)} samples")
print(f"\nFeatures: {feature_names}")
print(f"Classes: {list(class_names.values())}")

## 2. Train a Classifier

We'll train a Random Forest classifier. Case-explainer works with any classifier!

In [ ]:
# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.1%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=list(class_names.values())))

## 3. Create the CaseExplainer

Now we initialize the explainer with our training data. It builds a k-NN index for fast lookups.

In [ ]:
# Create explainer
explainer = CaseExplainer(
    X_train=X_train,
    y_train=y_train,
    feature_names=feature_names,
    class_names=class_names,
    algorithm='kd_tree',  # Fast for low-dimensional data
    scale_data=True       # Standardize features
)

print("\nExplainer initialized!")
print(f"Index built on {len(X_train)} training samples")

## 4. Explain a Single Prediction

Let's explain a test sample and see which training examples it resembles.

In [ ]:
# Pick a test sample
test_idx = 0
test_sample = X_test[test_idx]
true_label = y_test[test_idx]

# Get explanation
explanation = explainer.explain_instance(
    test_sample=test_sample,
    k=5,  # Find 5 nearest neighbors
    model=clf,
    true_class=true_label
)

# Print summary
print(explanation.summary())

### Understanding the Correspondence Score

The **correspondence** quantifies agreement between the prediction and nearest neighbors:

- **High (≥85%)**: Strong agreement, high confidence
- **Medium (70-85%)**: Moderate agreement
- **Low (<70%)**: Weak agreement, uncertain prediction

Let's examine the neighbors in detail:

In [ ]:
print(f"Prediction: {class_names[explanation.predicted_class]}")
print(f"True class: {class_names[explanation.true_class]}")
print(f"Correspondence: {explanation.correspondence:.1%} ({explanation.correspondence_interpretation})")
print(f"Correct: {explanation.is_correct()}")

print("\nNearest Neighbors:")
for i, neighbor in enumerate(explanation.neighbors, 1):
    print(f"\n{i}. Training sample #{neighbor.index}")
    print(f"   Distance: {neighbor.distance:.3f}")
    print(f"   Label: {class_names[neighbor.label]}")
    print(f"   Features: {neighbor.features}")

## 5. Visualize the Explanation

The plot shows neighbor distances colored by their class labels.

In [ ]:
# Create visualization
explanation.plot()
plt.tight_layout()
plt.show()

## 6. Batch Explanations

Let's explain all test samples and analyze the results.

In [ ]:
# Explain all test samples
explanations = explainer.explain_batch(
    X_test=X_test,
    k=5,
    y_test=y_test,
    model=clf
)

print(f"Generated {len(explanations)} explanations")

### Analyze Correspondence Distribution

In [ ]:
# Extract correspondence scores
correspondences = [exp.correspondence for exp in explanations]
correct_corr = [exp.correspondence for exp in explanations if exp.is_correct()]
incorrect_corr = [exp.correspondence for exp in explanations if not exp.is_correct()]

print("Correspondence Statistics:")
print(f"  Overall mean: {np.mean(correspondences):.1%}")
print(f"  Overall std: {np.std(correspondences):.1%}")
print(f"\n  Correct predictions: {np.mean(correct_corr):.1%}")
if incorrect_corr:
    print(f"  Incorrect predictions: {np.mean(incorrect_corr):.1%}")
else:
    print(f"  Incorrect predictions: None (perfect accuracy!)")

# Count by interpretation
high = sum(1 for exp in explanations if exp.correspondence_interpretation == 'high')
medium = sum(1 for exp in explanations if exp.correspondence_interpretation == 'medium')
low = sum(1 for exp in explanations if exp.correspondence_interpretation == 'low')

print(f"\nCorrespondence Levels:")
print(f"  High (≥85%): {high} samples")
print(f"  Medium (70-85%): {medium} samples")
print(f"  Low (<70%): {low} samples")

### Plot Correspondence Distribution

In [ ]:
# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(correspondences, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(np.mean(correspondences), color='red', linestyle='--', 
            label=f'Mean: {np.mean(correspondences):.1%}')
plt.xlabel('Correspondence Score')
plt.ylabel('Frequency')
plt.title('Distribution of Correspondence Scores')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Compare Correct vs Incorrect Predictions

In [ ]:
# Compare correspondence for correct vs incorrect
if incorrect_corr:
    plt.figure(figsize=(10, 6))
    plt.boxplot([correct_corr, incorrect_corr], labels=['Correct', 'Incorrect'])
    plt.ylabel('Correspondence Score')
    plt.title('Correspondence: Correct vs Incorrect Predictions')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nInsight: Correct predictions have {np.mean(correct_corr):.1%} correspondence")
    print(f"         Incorrect predictions have {np.mean(incorrect_corr):.1%} correspondence")
    print(f"         Difference: {np.mean(correct_corr) - np.mean(incorrect_corr):.1%}")
else:
    print("All predictions were correct! No incorrect predictions to compare.")

## 8. Inspect Low Correspondence Predictions

Predictions with low correspondence might be uncertain or unusual cases.

In [ ]:
# Find predictions with lowest correspondence
sorted_exps = sorted(explanations, key=lambda x: x.correspondence)
lowest_5 = sorted_exps[:5]

print("5 Predictions with Lowest Correspondence:\n")
for i, exp in enumerate(lowest_5, 1):
    print(f"{i}. Correspondence: {exp.correspondence:.1%} ({exp.correspondence_interpretation})")
    print(f"   Predicted: {class_names[exp.predicted_class]}")
    print(f"   True: {class_names[exp.true_class]}")
    print(f"   Correct: {exp.is_correct()}")
    print()

## Key Takeaways

1. **Case-based explanations** provide concrete training examples as evidence
2. **Correspondence** quantifies how well neighbors agree with the prediction
3. **High correspondence** indicates confident predictions backed by similar training data
4. **Low correspondence** suggests unusual cases or uncertain predictions
5. **Correct predictions** generally have higher correspondence than incorrect ones

## Next Steps

Try the other tutorials:
- `02_breast_cancer_tutorial.ipynb` - Medical diagnosis domain
- `03_fraud_detection_tutorial.ipynb` - Financial domain with imbalanced data
- `04_hardware_trojan_tutorial.ipynb` - Security domain with large-scale data